In [19]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv('train_set.csv')
df

,id_img,path,label
0,0,../data/images/train/angry\0.jpg,angry
1,1,../data/images/train/angry\1.jpg,angry
2,10,../data/images/train/angry\10.jpg,angry
3,10002,../data/images/train/angry\10002.jpg,angry
4,10016,../data/images/train/angry\10016.jpg,angry
...,...,...,...
28816,9969,../data/images/train/surprise\9969.jpg,surprise
28817,9985,../data/images/train/surprise\9985.jpg,surprise
28818,9990,../data/images/train/surprise\9990.jpg,surprise
28819,9992,../data/images/train/surprise\9992.jpg,surprise


In [35]:
test = pd.read_csv('test_set.csv')
test

,id_img
0,10052
1,10065
2,10079
3,10095
4,10121
...,...
7061,9806
7062,9830
7063,9853
7064,9878


In [4]:
df['label'].value_counts()

label
happy       7164
neutral     4982
sad         4938
fear        4103
angry       3993
surprise    3205
disgust      436
Name: count, dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['target'] = label_encoder.fit_transform(df['label'])

In [6]:
df['path'] = df['path'].str.replace('\\', '/')
df['path'] = df['path'].str.replace('../data/images/train/', './data/images/train/')

def cargar_y_convertir_imagen(ruta_imagen):
    image = cv2.imread(ruta_imagen, 0)
    image_array = np.array(image)
    return image_array

df['imagenes'] =df['path'].apply(cargar_y_convertir_imagen)

In [ ]:
df.head()

In [ ]:
df.sample(frac=1, random_state=42).reset_index(drop=True)

In [12]:
# Aplanar las imágenes y remodelarlas a 48x48
df['imagenes'] = df['imagenes'].apply(lambda x: np.array(x).reshape(48, 48))

# Separar características (X) y etiquetas (y)
X = np.stack(df['imagenes'].to_numpy())
y = df['target']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar las imágenes
X_train = X_train / 255.0  # Normalización
X_test = X_test / 255.0

# Construir el modelo de red neuronal
modelo = Sequential()
modelo.add(Reshape((48, 48, 1), input_shape=(48, 48)))  # Agrega una dimensión para los canales
modelo.add(Flatten())
modelo.add(Dense(128, activation='relu'))
modelo.add(Dense(64, activation='relu'))
modelo.add(Dense(7, activation='softmax'))  # 7 clases para 'angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'

# Compilar el modelo
modelo.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
modelo.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
accuracy = modelo.evaluate(X_test, y_test)[1]
print(f'Precisión del modelo en el conjunto de prueba: {accuracy}')

# Hacer predicciones en el conjunto de prueba
predicciones = modelo.predict(X_test)

# Puedes convertir las predicciones a etiquetas utilizando argmax
predicciones_labels = np.argmax(predicciones, axis=1)

Epoch 1/100
577/577 [==============================] - 2s 3ms/step - loss: 1.7998 - accuracy: 0.2632 - val_loss: 1.7189 - val_accuracy: 0.3190
Epoch 2/100
577/577 [==============================] - 2s 3ms/step - loss: 1.7128 - accuracy: 0.3190 - val_loss: 1.6713 - val_accuracy: 0.3467
Epoch 3/100
577/577 [==============================] - 2s 3ms/step - loss: 1.6852 - accuracy: 0.3365 - val_loss: 1.6601 - val_accuracy: 0.3465
Epoch 4/100
577/577 [==============================] - 2s 3ms/step - loss: 1.6621 - accuracy: 0.3475 - val_loss: 1.6539 - val_accuracy: 0.3417
Epoch 5/100
577/577 [==============================] - 2s 3ms/step - loss: 1.6459 - accuracy: 0.3515 - val_loss: 1.6433 - val_accuracy: 0.3632
Epoch 6/100
577/577 [==============================] - 2s 3ms/step - loss: 1.6315 - accuracy: 0.3645 - val_loss: 1.6323 - val_accuracy: 0.3569
Epoch 7/100
577/577 [==============================] - 2s 3ms/step - loss: 1.6193 - accuracy: 0.3687 - val_loss: 1.6967 - val_accuracy: 0.3281

In [ ]:
# Aplanar las imágenes y remodelarlas a 48x48
df['imagenes'] = df['imagenes'].apply(lambda x: np.array(x).reshape(48, 48, 1))

# Separar características (X) y etiquetas (y)
X = np.stack(df['imagenes'].to_numpy())
y = df['label_encoded']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar las imágenes
X_train = X_train / 255.0  # Normalización
X_test = X_test / 255.0

# Construir el modelo de red neuronal con capas convolucionales
modelo_cnn = Sequential()
modelo_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Conv2D(64, (3, 3), activation='relu'))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Flatten())
modelo_cnn.add(Dense(128, activation='relu'))
modelo_cnn.add(Dense(7, activation='softmax'))

# Compilar el modelo
modelo_cnn.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Imprimir el resumen del modelo
modelo_cnn.summary()

# Entrenar el modelo
modelo_cnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
accuracy_cnn = modelo_cnn.evaluate(X_test, y_test)[1]
print(f'Precisión del modelo CNN en el conjunto de prueba: {accuracy_cnn}')

In [16]:
# Aplanar las imágenes y remodelarlas a 48x48
df['imagenes'] = df['imagenes'].apply(lambda x: np.array(x).reshape(48, 48, 1))

# Separar características (X) y etiquetas (y)
X = np.stack(df['imagenes'].to_numpy())
y = df['target']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar las imágenes
X_train = X_train / 255.0  # Normalización
X_test = X_test / 255.0

# Construir el modelo de red neuronal con capas convolucionales
modelo_cnn = Sequential()
modelo_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Conv2D(64, (3, 3), activation='relu'))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Flatten())
modelo_cnn.add(Dense(128, activation='relu'))
modelo_cnn.add(Dense(7, activation='softmax'))

# Compilar el modelo
modelo_cnn.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Imprimir el resumen del modelo
modelo_cnn.summary()

# Entrenar el modelo
modelo_cnn.fit(X_train, y_train, epochs=6, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
accuracy_cnn = modelo_cnn.evaluate(X_test, y_test)[1]
print(f'Precisión del modelo CNN en el conjunto de prueba: {accuracy_cnn}')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 6400)              0         
                                                                 
 dense_16 (Dense)            (None, 128)              

In [34]:
df['imagenes'] = df['imagenes'].apply(lambda x: np.array(x).reshape(48, 48, 1))

# Separar características (X) y etiquetas (y)
X = np.stack(df['imagenes'].to_numpy())
y = df['target']


# Normalizar las imágenes
X = X / 255.0 


# Construir el modelo de red neuronal con capas convolucionales
modelo_cnn = Sequential()
modelo_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Conv2D(64, (3, 3), activation='relu'))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Flatten())
modelo_cnn.add(Dense(128, activation='relu'))
modelo_cnn.add(Dense(7, activation='softmax'))

# Compilar el modelo
modelo_cnn.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Imprimir el resumen del modelo
modelo_cnn.summary()

# Definir EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenar el modelo con EarlyStopping
historia = modelo_cnn.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
accuracy_cnn = modelo_cnn.evaluate(X, y)[1]
print(f'Precisión del modelo CNN en el conjunto de prueba: {accuracy_cnn}')

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 23, 23, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 10, 10, 64)        0         
 ng2D)                                                           
                                                                 
 flatten_9 (Flatten)         (None, 6400)              0         
                                                                 
 dense_22 (Dense)            (None, 128)              

In [37]:
from tensorflow.keras.models import save_model

modelo_cnn.save('modelo_entrenado2.h5')

c:\Users\migue\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [36]:
test['path'] = test['id_img'].apply(lambda x: f'./data/images/test/{x}.jpg')

In [38]:
test['imagenes'] = test['path'].apply(cargar_y_convertir_imagen)

In [39]:
test

,id_img,path,imagenes
0,10052,./data/images/test/10052.jpg,"[[58, 66, 70, 77, 117, 154, 137, 108, 76, 70, ..."
1,10065,./data/images/test/10065.jpg,"[[23, 26, 21, 9, 6, 19, 33, 11, 3, 63, 89, 73,..."
2,10079,./data/images/test/10079.jpg,"[[201, 182, 182, 184, 205, 204, 203, 220, 223,..."
3,10095,./data/images/test/10095.jpg,"[[93, 86, 78, 78, 80, 92, 109, 99, 104, 107, 1..."
4,10121,./data/images/test/10121.jpg,"[[11, 6, 1, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 2, 3..."
...,...,...,...
7061,9806,./data/images/test/9806.jpg,"[[255, 253, 255, 255, 253, 255, 255, 250, 254,..."
7062,9830,./data/images/test/9830.jpg,"[[84, 71, 70, 68, 60, 47, 53, 45, 51, 58, 56, ..."
7063,9853,./data/images/test/9853.jpg,"[[250, 253, 253, 252, 252, 252, 253, 251, 234,..."
7064,9878,./data/images/test/9878.jpg,"[[228, 224, 227, 202, 45, 8, 6, 8, 15, 21, 19,..."


In [40]:
from tensorflow.keras.models import load_model

# Cargar el modelo entrenado
modelo_cnn = load_model('modelo_entrenado2.h5')  # Reemplaza 'modelo_cnn.h5' con el nombre de tu modelo guardado

# Aplanar las imágenes y remodelarlas a 48x48
test['imagenes'] = test['imagenes'].apply(lambda x: np.array(x).reshape(48, 48, 1))

# Normalizar las imágenes
test['imagenes'] = test['imagenes'] / 255.0

# Hacer predicciones en el conjunto de prueba
X_test2 = np.stack(test['imagenes'].to_numpy())
predicciones2 = modelo_cnn.predict(X_test)

# Obtener las etiquetas predichas
predicciones_labels2 = np.argmax(predicciones, axis=1)

# Agregar las etiquetas predichas al DataFrame test
test['label_encoded_pred'] = predicciones_labels2

221/221 [==============================] - 1s 5ms/step


In [41]:
test.drop(columns=(['path', 'imagenes']), inplace=True)

In [52]:
test

,id_img,label_encoded_pred,label
0,10052,4,neutral
1,10065,2,fear
2,10079,4,neutral
3,10095,5,sad
4,10121,0,angry
...,...,...,...
7061,9806,6,surprise
7062,9830,2,fear
7063,9853,0,angry
7064,9878,4,neutral


In [51]:
test['label'] = label_encoder.inverse_transform(predicciones_labels2)

In [53]:
submission = pd.DataFrame({'id_img': test['id_img'], 'label': test['label']})

submission.to_csv('submission.csv', index=False)

In [54]:
submission

,id_img,label
0,10052,neutral
1,10065,fear
2,10079,neutral
3,10095,sad
4,10121,angry
...,...,...
7061,9806,surprise
7062,9830,fear
7063,9853,angry
7064,9878,neutral


In [ ]:
submission